# 0 Setting

In [1]:
# Parameter Setting
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
folder_name = 'Research-(D5) Synthesized input model'
pretrained_model_name = 'model_microsoft.ckpt'

config = {
    'learning_rate': 4e-3,
    'batch_size': 32,
    'seq_length': 5,

    'shuffle': False,
    'criterion': torch.nn.CrossEntropyLoss(),
    'seed': 42,

    'n_epochs': 3000,
    'early_stop': 500,
    'device': device,
}


sector_id_list = [ # Not confirmed
    "XLK",  # Information Technology
    "XLV",  # Health Care
    "XLF",  # Financials
    "XLI",  # Industrials
    "XLY",  # Consumer Discretionary
    "XLE",  # Energy
    "XLB",  # Materials
    "XLC",  # Communication Services
    "XLU",  # Utilities
    "XLRE",  # Real Estate
    "XLP"  # Consumer Staples
]

company_list = [
    "Technology",
    "Health Care",
    "Financials",
    "Industrials",
    "Consumer Discretionary",
    "Energy",
    "Materials",
    "Communication Services",
    "Utilities",
    "Real Estate",
    "Consumer Staples"
]

process_id = 2  #26

company_name = company_list[process_id]

config_2 = {'input_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/data/2_'+company_name+'_for_model.csv',
            'save_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/model_saved/model_'+company_name+'.ckpt',
            # 'pretrained_model_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/premodel/' + pretrained_model_name,
            # 'continue_model_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/model_saved/model_1.ckpt'
            }

feature = [
    # X_1
    # 'input_ids',
    # 'attention_mask',
    'section_dummy',
    'publication_dummy',
    'input_ids_bert',
    'attention_mask_bert',

    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'today_return_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',
    # 'sentiment',

    # 'logit0_bert',
    # 'logit1_bert',
    # 'logit0_finbert',
    # 'logit1_finbert',
    # 'logit2_finbert',

    # 'logit0_bert_average',
    # 'logit1_bert_average',
    # 'logit0_finbert_average',
    # 'logit1_finbert_average',
    # 'logit2_finbert_average',

    # 2. market index
    '^DJI',
    '^GSPC',
    '^NDX',
    '^IXIC',
    '^SOX',
    '^NYA',

    # y
    # '1_day_return',
    # '2_day_return',
    # '3_day_return',
    # '4_day_return',
    # '5_day_return',
    # '1_day_return_cate',
    # '2_day_return_cate',
    # '3_day_return_cate',
    # '4_day_return_cate',
    # '5_day_return_cate',
    # '^DJI', '^DJI_1_day_return', '^GSPC', '^GSPC_1_day_return',
    #    '^NDX', '^NDX_1_day_return', '^IXIC', '^IXIC_1_day_return', '^SOX',
    #    '^SOX_1_day_return',
    # 'excess_return_^DJI',
    # 'excess_return_^DJI_cate',
    # 'excess_return_^GSPC',
    'excess_return_^GSPC_cate',
    # 'excess_return_^NDX',
    # 'excess_return_^NDX_cate',
    # 'excess_return_^IXIC',
    # 'excess_return_^IXIC_cate',
    # 'excess_return_^SOX',
    # 'excess_return_^SOX_cate',

    # Do not mark the datetime, it's for operation
    'datetime',
    ]

# All the news dataset
# time_start = '2016-01-01T00:00:00'
# time_end = '2020-04-02T00:00:00'

time_start = '2016-01-01T00:00:00'
time_end = '2019-12-31T00:00:00'

print(len(feature)-2)

11


## (1) Import

In [2]:
# Google
from google.colab import drive
drive.mount('/content/drive')

# pip installation
!pip install transformers

# Basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# PyTorch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from transformers import BertTokenizer, BertModel, BertConfig

# others
from datetime import datetime, timedelta
from tqdm import tqdm
from torchsummary import summary
import ast

Mounted at /content/drive


In [3]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed for reproducibility
same_seed(config['seed'])


In [4]:
df = pd.read_csv(config_2['input_path'])
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Capital Gains', 'today_return', 'today_return_cate', '1_day_return',
       '2_day_return', '3_day_return', '4_day_return', '5_day_return',
       '1_day_return_cate', '2_day_return_cate', '3_day_return_cate',
       '4_day_return_cate', '5_day_return_cate', 'Sma', 'Rsi', 'Kd', 'Ema_12',
       'Ema_26', 'Macd', 'datetime', '^DJI', '^DJI_1_day_return', '^GSPC',
       '^GSPC_1_day_return', '^NDX', '^NDX_1_day_return', '^IXIC',
       '^IXIC_1_day_return', '^SOX', '^SOX_1_day_return', '^NYA',
       '^NYA_1_day_return', 'excess_return_^DJI', 'excess_return_^DJI_cate',
       'excess_return_^GSPC', 'excess_return_^GSPC_cate', 'excess_return_^NDX',
       'excess_return_^NDX_cate', 'excess_return_^IXIC',
       'excess_return_^IXIC_cate', 'excess_return_^SOX',
       'excess_return_^SOX_cate', 'excess_return_^NYA',
       'excess_return_^NYA_cate', 'input_ids', 'attention_mask',
       'section_dummy', '

In [5]:
df = df.sort_values(by='datetime', ascending=True)
df.shape

(1070, 68)

In [6]:
# Only contain selected features
df = df[feature]
df.columns

Index(['section_dummy', 'publication_dummy', 'input_ids_bert',
       'attention_mask_bert', 'today_return', '^DJI', '^GSPC', '^NDX', '^IXIC',
       '^SOX', '^NYA', 'excess_return_^GSPC_cate', 'datetime'],
      dtype='object')

In [7]:
# Rename input_ids_bert to input_ids
df = df.rename(columns={'input_ids_bert': 'input_ids', 'attention_mask_bert': 'attention_mask'})

In [8]:
feature[2] = 'input_ids'
feature[3] = 'attention_mask'
feature

['section_dummy',
 'publication_dummy',
 'input_ids',
 'attention_mask',
 'today_return',
 '^DJI',
 '^GSPC',
 '^NDX',
 '^IXIC',
 '^SOX',
 '^NYA',
 'excess_return_^GSPC_cate',
 'datetime']

## (2) check nan

In [9]:
df[df.isna().any(axis=1)]

,section_dummy,publication_dummy,input_ids,attention_mask,today_return,^DJI,^GSPC,^NDX,^IXIC,^SOX,^NYA,excess_return_^GSPC_cate,datetime
1068,"[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, ...",NaN,NaN,-0.008621,-0.013373,-0.011041,-0.012725,-0.013261,-0.018769,-0.007306,1,2020-04-01
1069,"[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, ...",NaN,NaN,0.031395,0.028530,0.027805,0.025488,0.023213,0.040980,0.025175,0,2020-04-02


In [10]:
df.isnull().sum()

section_dummy               0
publication_dummy           0
input_ids                   2
attention_mask              2
today_return                0
^DJI                        0
^GSPC                       0
^NDX                        0
^IXIC                       0
^SOX                        0
^NYA                        0
excess_return_^GSPC_cate    0
datetime                    0
dtype: int64

In [11]:
df = df.dropna()
df = df.reset_index(drop=True)
df.isnull().sum()

section_dummy               0
publication_dummy           0
input_ids                   0
attention_mask              0
today_return                0
^DJI                        0
^GSPC                       0
^NDX                        0
^IXIC                       0
^SOX                        0
^NYA                        0
excess_return_^GSPC_cate    0
datetime                    0
dtype: int64

## (2) Time Period Selection

In [12]:
# We use index to filter for time periods
df = df[(df['datetime']> time_start) & (df['datetime'] < time_end)]

# Drop datetime after using it
df.drop(columns=['datetime'], inplace=True)
df.shape

(1006, 12)

## (3) Transform str back to list

In [13]:
# 将字符串转换回列表的函数
def string_to_list(s):
    return ast.literal_eval(s)

# 将列中的字符串转换回列表
df['input_ids'] = df['input_ids'].apply(string_to_list)
df['attention_mask'] = df['attention_mask'].apply(string_to_list)
df['section_dummy'] = df['section_dummy'].apply(string_to_list)
df['publication_dummy'] = df['publication_dummy'].apply(string_to_list)

In [14]:
df['input_ids'][0][0]

[101,
 2149,
 15768,
 1011,
 2813,
 2358,
 27790,
 2015,
 2044,
 5410,
 2707,
 2000,
 2095,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [15]:
df['attention_mask'][0][0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [16]:
df['section_dummy'][0][0]

[0, 0, 1, 0, 0, 0, 0, 0, 0]

## (3) List: Same amount of elements

In [17]:
input_ids_list_length = len(df['input_ids'][0][0])
attention_mask_list_length = len(df['attention_mask'][0][0])
section_dummy_list_length = len(df['section_dummy'][0][0])
publication_dummy_list_length = len(df['publication_dummy'][0][0])

In [18]:
# 找到最大的內部列表長度
max_inner_length = max(df['input_ids'].apply(len))

# 定義一個函數來填充內部列表，使其長度達到最大值
def pad_inner_list(lst, zero_list):
    while len(lst) < max_inner_length:
        lst.append(zero_list)  # 這裡可以填充任何你想要的值，例如 None

# 將 "input_ids" 列中的每個內部列表填充到最大長度
df['input_ids'].apply(pad_inner_list, zero_list=[0] * input_ids_list_length)
df['attention_mask'].apply(pad_inner_list, zero_list=[0] * attention_mask_list_length)
df['section_dummy'].apply(pad_inner_list, zero_list=[0] * section_dummy_list_length)
df['publication_dummy'].apply(pad_inner_list, zero_list=[0] * publication_dummy_list_length)
df

,section_dummy,publication_dummy,input_ids,attention_mask,today_return,^DJI,^GSPC,^NDX,^IXIC,^SOX,^NYA,excess_return_^GSPC_cate
0,"[[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 0, ...","[[101, 2149, 15768, 1011, 2813, 2358, 27790, 2...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...",0.001285,-0.014739,-0.012531,0.002854,0.001111,0.008979,-0.001845,1
1,"[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, ...","[[101, 2220, 2693, 2869, 1024, 1061, 6806, 208...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",0.002136,0.000651,0.001455,-0.006175,-0.005370,-0.012003,0.002104,1
2,"[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, ...","[[101, 2149, 3514, 27221, 2091, 3963, 16653, 1...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",-0.000865,-0.014475,-0.010663,0.007758,0.004570,-0.011092,-0.003948,1
3,"[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 0, ...","[[101, 3725, 1005, 1055, 3027, 3366, 14590, 21...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",-0.007516,-0.022161,-0.021271,-0.009984,-0.009917,-0.010667,-0.010350,0
4,"[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 0, ...","[[101, 10666, 20116, 12162, 2891, 1010, 6221, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,...",-0.024283,-0.010456,-0.012302,-0.015069,-0.016601,-0.019801,-0.016923,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1001,"[[1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, ...","[[101, 13371, 1011, 2501, 3784, 4341, 2507, 10...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",0.000326,-0.001999,-0.000642,-0.000665,-0.000238,0.001204,-0.000562,1
1002,"[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, ...","[[101, 2053, 2240, 2000, 2604, 2797, 9924, 101...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",0.004559,0.002871,0.003938,0.006924,0.005817,-0.000499,0.002103,0
1003,"[[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, ...","[[101, 17765, 22818, 12609, 19236, 1024, 2053,...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,...",-0.004207,-0.001049,-0.002220,-0.003961,-0.004735,-0.006431,-0.002340,0
1004,"[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 0, ...","[[101, 3795, 6089, 1011, 15768, 2203, 10476, 2...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...",-0.007441,-0.006722,-0.005802,-0.006463,-0.006492,-0.006593,-0.005384,0


In [19]:
df['section_dummy'][0]

[[0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [20]:
# 使用 apply 方法計算每個列表中元素的數量
# Note: This number is not about tokens. It's about the number of news in that day.
temp = df['input_ids'].apply(len)

# 打印 DataFrame
temp

0       15
1       15
2       15
3       15
4       15
        ..
1001    15
1002    15
1003    15
1004    15
1005    15
Name: input_ids, Length: 1006, dtype: int64

In [21]:
# 計算"input_ids"列中所有list的平均長度
average_length = df['input_ids'].apply(len).mean()

# 計算"input_ids"列中最長的list的長度
max_length = df['input_ids'].apply(len).max()

# 計算"input_ids"列中最短的list的長度
min_length = df['input_ids'].apply(len).min()

# 打印結果
print(f"平均長度: {average_length}")
print(f"最長長度: {max_length}")
print(f"最短長度: {min_length}")


平均長度: 15.0
最長長度: 15
最短長度: 15


## int to float (section, publication)

In [22]:
type(df.input_ids[0][0])

list

In [23]:
def recursive_convert_to_float(item):
    if isinstance(item, list):
        return [recursive_convert_to_float(x) if x is not None else None for x in item]
    else:
        return float(item) if item is not None else None

# 使用 apply 方法將函數應用於每個元素
df['section_dummy'] = df['section_dummy'].apply(recursive_convert_to_float)
df['publication_dummy'] = df['publication_dummy'].apply(recursive_convert_to_float)

In [24]:
df['section_dummy'][0]

[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

## (4) Train_test_split

In [25]:
# 1. Set up X, y
to_remove_list = ['datetime', 'excess_return_^GSPC_cate']

# Filter out values in to_remove_list
filtered_list = [x for x in feature if x not in to_remove_list]

X = df[filtered_list]
y = df['excess_return_^GSPC_cate']

In [26]:
# Check X, y shape
print('X:', X.shape)
print('y:', y.shape)

X: (1006, 11)
y: (1006,)


In [27]:
# 2. train_test_split
# val dataset for final examination
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=config['seed'], shuffle=config['shuffle'])
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=config['seed'], shuffle=config['shuffle'])

# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=config['seed'], shuffle=config['shuffle'])
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.66, random_state=config['seed'], shuffle=config['shuffle'])
X_train


,section_dummy,publication_dummy,input_ids,attention_mask,today_return,^DJI,^GSPC,^NDX,^IXIC,^SOX,^NYA
0,"[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[101, 2149, 15768, 1011, 2813, 2358, 27790, 2...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...",0.001285,-0.014739,-0.012531,0.002854,0.001111,0.008979,-0.001845
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[101, 2220, 2693, 2869, 1024, 1061, 6806, 208...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",0.002136,0.000651,0.001455,-0.006175,-0.005370,-0.012003,0.002104
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[101, 2149, 3514, 27221, 2091, 3963, 16653, 1...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",-0.000865,-0.014475,-0.010663,0.007758,0.004570,-0.011092,-0.003948
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[101, 3725, 1005, 1055, 3027, 3366, 14590, 21...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",-0.007516,-0.022161,-0.021271,-0.009984,-0.009917,-0.010667,-0.010350
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[101, 10666, 20116, 12162, 2891, 1010, 6221, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,...",-0.024283,-0.010456,-0.012302,-0.015069,-0.016601,-0.019801,-0.016923
...,...,...,...,...,...,...,...,...,...,...,...
598,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[101, 5650, 6291, 1010, 2726, 4297, 2953, 356...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,...",-0.008162,0.000298,-0.001612,-0.001512,-0.001358,-0.003253,-0.001389
599,"[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[101, 10651, 1015, 1011, 2148, 3060, 14566, 8...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,...",0.001423,0.005234,-0.000870,-0.002137,-0.001661,-0.004453,0.002540
600,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[101, 18411, 8004, 28208, 2048, 18288, 2005, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,...",0.006035,-0.008509,-0.005076,-0.005924,-0.005712,-0.001986,-0.005106
601,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[101, 10651, 1015, 1011, 1050, 1012, 4420, 22...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",0.000355,0.005215,0.007115,0.015533,0.012397,0.015664,0.003293


## (5) Scaler

In [28]:
scale_feature = [
    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'Today_trend_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',
    # 'sentiment',

    # 'logit0_bert',
    # 'logit1_bert',
    # 'logit0_finbert',
    # 'logit1_finbert',
    # 'logit2_finbert',

    # 'logit0_bert_average',
    # 'logit1_bert_average',
    # 'logit0_finbert_average',
    # 'logit1_finbert_average',
    # 'logit2_finbert_average',

    # 2. market index
    '^DJI',
    '^GSPC',
    '^NDX',
    '^IXIC',
    '^SOX',
    '^NYA',

    # 'datetime'
    ]

def CustomScaler(X_train, X_val, X_test):
  scaler = MinMaxScaler()
  for i in scale_feature:

    # 對特定欄位進行標準化
    X_train_scaled = scaler.fit_transform(X_train[[i]])
    X_val_scaled = scaler.transform(X_val[[i]])
    X_test_scaled = scaler.transform(X_test[[i]])

    # 將標準化後的值重新賦值給 DataFrame
    X_train[i] = X_train_scaled
    X_val[i] = X_val_scaled
    X_test[i] = X_test_scaled

  return X_train, X_val, X_test

X_train, X_val, X_test = CustomScaler(X_train, X_val, X_test)

X_train

,section_dummy,publication_dummy,input_ids,attention_mask,today_return,^DJI,^GSPC,^NDX,^IXIC,^SOX,^NYA
0,"[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[101, 2149, 15768, 1011, 2813, 2358, 27790, 2...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...",0.534311,0.355624,0.377236,0.525722,0.515721,0.591670,0.548185
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[101, 2220, 2693, 2869, 1024, 1061, 6806, 208...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",0.543987,0.557596,0.578553,0.424253,0.436452,0.374338,0.606119
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[101, 2149, 3514, 27221, 2091, 3963, 16653, 1...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",0.509845,0.359087,0.404126,0.580837,0.558035,0.383766,0.517342
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[101, 3725, 1005, 1055, 3027, 3366, 14590, 21...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",0.434182,0.258224,0.251419,0.381444,0.380842,0.388167,0.423407
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[101, 10666, 20116, 12162, 2891, 1010, 6221, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,...",0.243421,0.411837,0.380522,0.324291,0.299087,0.293563,0.326980
...,...,...,...,...,...,...,...,...,...,...,...
598,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[101, 5650, 6291, 1010, 2726, 4297, 2953, 356...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,...",0.426829,0.552968,0.534407,0.476651,0.485527,0.464971,0.554878
599,"[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[101, 10651, 1015, 1011, 2148, 3060, 14566, 8...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,...",0.535876,0.617738,0.545086,0.469628,0.481824,0.452541,0.612522
600,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[101, 18411, 8004, 28208, 2048, 18288, 2005, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,...",0.588346,0.437380,0.484541,0.427071,0.432268,0.478096,0.500349
601,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[101, 10651, 1015, 1011, 1050, 1012, 4420, 22...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",0.523729,0.617488,0.660028,0.668223,0.653762,0.660923,0.623563


## (6) Check number

In [29]:
def calculate_class_stats(y):
    class_counts = y.value_counts()
    total_samples = len(y)
    class_ratios = class_counts / total_samples
    return class_counts, class_ratios

# 計算類別數量和比例
train_class_counts, train_class_ratios = calculate_class_stats(y_train)
val_class_counts, val_class_ratios = calculate_class_stats(y_val)
test_class_counts, test_class_ratios = calculate_class_stats(y_test)

# 創建包含數量和比例的 DataFrame
class_stats = pd.DataFrame({
    'Train Count': train_class_counts,
    'Train Ratio': train_class_ratios,
    'Validation Count': val_class_counts,
    'Validation Ratio': val_class_ratios,
    'Test Count': test_class_counts,
    'Test Ratio': test_class_ratios
})

# 打印 DataFrame
print(class_stats)


   Train Count  Train Ratio  Validation Count  Validation Ratio  Test Count  \
0          315     0.522388               106          0.527363          87   
1          288     0.477612                95          0.472637         115   

   Test Ratio  
0    0.430693  
1    0.569307  


In [30]:
# Time period
print('Time Period')
print('From:', time_start)
print('To:', time_end, '\n')

# Sample size
print('Sample size:', X.shape[0])
print('Feature:', X.columns, '\n')
print('Target:', y.name, '\n')
print('Train: Val: Test =', X_train.shape[0], X_val.shape[0], X_test.shape[0])

Time Period
From: 2016-01-01T00:00:00
To: 2019-12-31T00:00:00 

Sample size: 1006
Feature: Index(['section_dummy', 'publication_dummy', 'input_ids', 'attention_mask',
       'today_return', '^DJI', '^GSPC', '^NDX', '^IXIC', '^SOX', '^NYA'],
      dtype='object') 

Target: excess_return_^GSPC_cate 

Train: Val: Test = 603 201 202


# Model

## (1) Dataset & Dataloader

In [31]:
# Dataset
X_1 =['input_ids', 'attention_mask', 'section_dummy', 'publication_dummy']


class CustomDataset(Dataset):
    def __init__(self, X, y, config):
        # X_1
        self.input_ids = X['input_ids']
        self.attention_mask = X['attention_mask']

        # X_2
        self.X_2 = torch.tensor(X.drop(columns=X_1).values, dtype=torch.float)

        # y
        self.y = torch.tensor(y.values, dtype=torch.long)

        # other setting
        self.len = X.shape[0]
        self.seq_length = config['seq_length']

    def __getitem__(self,idx):
        # X_1
        input_ids_list = self.input_ids[idx : idx + self.seq_length].tolist() # All to list
        input_ids = torch.tensor(input_ids_list) # Then to tensor
        attention_mask_list = self.attention_mask[idx : idx + self.seq_length].tolist()
        attention_mask = torch.tensor(attention_mask_list)

        # X_2
        X_2 = self.X_2[idx : idx + self.seq_length]

        # 3. y
        y = self.y[idx + self.seq_length - 1]

        return input_ids, attention_mask, X_2, y

    def __len__(self):
        return self.len - self.seq_length

In [32]:
# DataLoader
train_dataset = CustomDataset(X_train, y_train, config)
val_dataset = CustomDataset(X_val, y_val, config)
test_dataset = CustomDataset(X_test, y_test, config)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], drop_last=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], drop_last=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], drop_last=True, pin_memory=True)

# Check loader output
for batch in train_loader:
    input_ids, attention_mask, X_2, y = batch

    # 打印批次数据的形状，以确保它们符合预期
    print("Input IDs shape:", input_ids.shape)
    print("Attention Mask shape:", attention_mask.shape)
    print("X_2 shape:", X_2.shape)
    print("Labels shape:", y.shape)

    # print("Input IDs:", input_ids)
    # print("Attention Mask:", attention_mask)
    # print("Section:", section)
    # print("Publication:", publication)
    # print("X_2:", X_2)
    # print("Labels:", y)

    break


Input IDs shape: torch.Size([32, 5, 15, 32])
Attention Mask shape: torch.Size([32, 5, 15, 32])
X_2 shape: torch.Size([32, 5, 7])
Labels shape: torch.Size([32])


## (2) Model Architecture

### 0 Param setting

In [33]:
new_config = {
    'h_text_size': 5,
    'h_c_size': 1,
}

config.update(new_config)

### 1 New design

In [34]:
# New method: C as a scaler + activation function + layer normalization
class MyModel(nn.Module):
    def __init__(self, base_model, config, element_size, X_2_length, batch_size):
        super(MyModel, self).__init__()
        self.seq_length = config['seq_length']
        self.batch_size = batch_size
        self.element_size = element_size
        self.abandon_tensor = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                              0, 0, 0, 0, 0, 0, 0, 0], device=device)
        self.config = config

        # 1. News
        # text
        self.base_model = base_model
        self.fc1 = nn.Linear(768, config['h_text_size'])

        # 2. LSTM
        self.lstm_1 = nn.LSTM(config['h_text_size'] + X_2_length, 32, dropout=0.1, num_layers=2, batch_first=True, bidirectional=False)
        self.sequential = nn.Sequential(
            nn.Linear(32, 2)
        )
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.batch_norm = nn.BatchNorm1d(config['h_text_size'])



    def forward(self, input_ids, attention_mask, X_2):
        # 1. BERT
        flattened_input_ids = input_ids.view(-1, 32)
        flattened_attention_mask = attention_mask.view(-1, 32)

        e_list = []
        for i in range(0, flattened_input_ids.size(0), self.element_size):
          sub_input_ids = flattened_input_ids[i:i+self.element_size]
          sub_attention_mask = flattened_attention_mask[i:i+self.element_size]

          non_zero_mask = (sub_input_ids != 0).any(dim=1)
          non_zero_input_ids = sub_input_ids[non_zero_mask]
          non_zero_attention_mask = sub_attention_mask[non_zero_mask]

          # input_ids, attention_mask
          out = self.base_model(input_ids=non_zero_input_ids, attention_mask=non_zero_attention_mask)
          out = out.pooler_output
          h_text = self.fc1(out)

          batch_norm_h = self.batch_norm(h_text)
          element_mean = torch.mean(batch_norm_h, dim=0)
          e_list.append(element_mean)

        temp_tensor = torch.stack(e_list)
        b_tensor = temp_tensor.view(self.batch_size, self.seq_length, self.config['h_text_size'])

        # After BERT

        h_tech = X_2

        h = torch.cat([b_tensor, h_tech], dim=2)

        # 3. LSTM
        out, _ = self.lstm_1(h)
        out = out[:, -1, :]  # Get the last one of LSTM output for prediction of next-term

        final_out = self.sequential(out)

        return final_out


## (4) Load Model

### 1. Load pretrain model

In [35]:
# 載入預訓練模型
from transformers import AutoModelForSequenceClassification
# base_model = PreModel(base_model)
# base_model.load_state_dict(torch.load(config_2['pretrained_model_path']))
# bert_config = BertConfig(hidden_dropout_prob=0.2)
# model_name = 'ProsusAI/finbert'
model_name = 'bert-base-uncased'
base_model = BertModel.from_pretrained(model_name)
# base_model = BertModel.from_pretrained(model_name, config=bert_config)
# base_model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Parameter
element_size = len(df['input_ids'][0])  # 114
X_2_length = len(feature) - 6


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

### 2. Initiate Model

In [36]:
model = MyModel(base_model, config, element_size, X_2_length, config['batch_size'])

model.to(device)

MyModel(
  (base_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

### Extra: Contunue training

In [37]:
# model = MyModel(base_model, config, section_length, publication_length, X_2_length)
# model.load_state_dict(torch.load(config_2['save_path']))
# model.to(device)

# # 分段訓練
# trainer2(model, train_loader, val_loader, config, device)
# trainer1(model, train_loader, val_loader, config, device)

## (5) Require_grad

In [38]:
# Freeze all layers
for param in model.base_model.parameters():
  param.requires_grad = False

# Unfreeze part of layers
# for param in model.base_model.encoder.layer[6].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[7].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[8].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[9].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[10].parameters():
#     param.requires_grad = True

for param in model.base_model.encoder.layer[11].parameters():
    param.requires_grad = True

# for param in model.base_model.bert.encoder.layer[11].parameters():
#     param.requires_grad = True

# for param in model.base_model.classifier.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc1.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc2.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc3.parameters():
    # param.requires_grad = True

# Check requires_grad status
for name, param in model.named_parameters():
    print(name, param.requires_grad)

base_model.embeddings.word_embeddings.weight False
base_model.embeddings.position_embeddings.weight False
base_model.embeddings.token_type_embeddings.weight False
base_model.embeddings.LayerNorm.weight False
base_model.embeddings.LayerNorm.bias False
base_model.encoder.layer.0.attention.self.query.weight False
base_model.encoder.layer.0.attention.self.query.bias False
base_model.encoder.layer.0.attention.self.key.weight False
base_model.encoder.layer.0.attention.self.key.bias False
base_model.encoder.layer.0.attention.self.value.weight False
base_model.encoder.layer.0.attention.self.value.bias False
base_model.encoder.layer.0.attention.output.dense.weight False
base_model.encoder.layer.0.attention.output.dense.bias False
base_model.encoder.layer.0.attention.output.LayerNorm.weight False
base_model.encoder.layer.0.attention.output.LayerNorm.bias False
base_model.encoder.layer.0.intermediate.dense.weight False
base_model.encoder.layer.0.intermediate.dense.bias False
base_model.encoder.la

# Training

save based on acc and loss

In [39]:
def trainer(model, train_loader, val_loader, config, device):

    criterion = config['criterion']
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])

    writer = SummaryWriter()  # Writer of tensoboard.
    n_epochs, best_loss, best_acc, step, early_stop_count = config['n_epochs'], math.inf, 0, 0, 0

    # 1. Training
    for epoch in range(n_epochs):
      model.train()  # Set the model to training mode
      loss_record = []

      train_pbar = tqdm(train_loader, position=0, leave=True)  # tqdm is a package to visualize your training progress.
      for input_ids, attention_mask, X_2, y in train_loader:
        optimizer.zero_grad()  # Set gradient to zero

        # Forward pass
        input_ids, attention_mask, X_2, y = input_ids.to(device), attention_mask.to(device), X_2.to(device), y.to(device)
        pred = model(input_ids, attention_mask, X_2)
        loss = criterion(pred, y)
        loss.backward()                     # Compute gradient(backpropagation).
        optimizer.step()                    # Update parameters.
        step += 1
        loss_record.append(loss.detach().item())

        # Display current epoch number and loss on tqdm progress bar.
        train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
        train_pbar.set_postfix({'loss': loss.detach().item()})

      mean_train_loss = sum(loss_record)/len(loss_record)
      writer.add_scalar('Loss/train', mean_train_loss, step)

      # 2. Evaluation
      model.eval() # Set your model to evaluation mode.
      loss_record = []
      predicted_labels_list = []
      targets_list = []
      for input_ids, attention_mask, X_2, y in val_loader:
          input_ids, attention_mask, X_2, y = input_ids.to(device), attention_mask.to(device), X_2.to(device), y.to(device)
          with torch.no_grad():
              pred = model(input_ids, attention_mask, X_2)
              _, predicted = torch.max(pred, 1)
              loss = criterion(pred, y)
              predicted_labels_list.extend(predicted.tolist())
              targets_list.extend(y.tolist())
              loss_record.append(loss.item())
      accuracy = accuracy_score(targets_list, predicted_labels_list)

      # Mean
      mean_valid_loss = sum(loss_record)/len(loss_record)
      print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}, Val Acc: {accuracy:.4f}')
      writer.add_scalar('Loss/valid', mean_valid_loss, step)

      # 3. Judge of saving model
      if (accuracy > best_acc) or ((accuracy == best_acc) and (mean_valid_loss < best_loss)):
          best_acc = accuracy
          best_loss = mean_valid_loss
          torch.save(model.state_dict(), config_2['save_path']) # Save your best model
          print('Saving model with loss {:.3f}...'.format(best_acc))
          early_stop_count = 0
      else:
          early_stop_count += 1

      if early_stop_count >= config['early_stop']:
          print('\nModel is not improving, so we halt the training session.')
          return




In [ ]:
# 全部訓練
trainer(model, train_loader, val_loader, config, device)


Epoch [1/3000]:   0%|          | 0/18 [01:00<?, ?it/s, loss=0.685]

Epoch [1/3000]: Train loss: 0.6949, Valid loss: 0.6924, Val Acc: 0.5208
Saving model with loss 0.521...


Epoch [2/3000]:   0%|          | 0/18 [00:45<?, ?it/s, loss=0.677]

Epoch [2/3000]: Train loss: 0.6922, Valid loss: 0.6923, Val Acc: 0.5208
Saving model with loss 0.521...


Epoch [3/3000]:   0%|          | 0/18 [00:45<?, ?it/s, loss=0.677]

...

Epoch [13/3000]:   0%|          | 0/18 [00:44<?, ?it/s, loss=0.677]

Epoch [13/3000]: Train loss: 0.6922, Valid loss: 0.6923, Val Acc: 0.5208
Saving model with loss 0.521...


Epoch [14/3000]:   0%|          | 0/18 [00:47<?, ?it/s, loss=0.677]

Epoch [14/3000]: Train loss: 0.6923, Valid loss: 0.6923, Val Acc: 0.5208


Epoch [15/3000]:   0%|          | 0/18 [00:42<?, ?it/s, loss=0.697]

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

# Evaluate

In [ ]:
# Evaluation Dataloader
con_train_loader = DataLoader(train_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)
con_val_loader = DataLoader(val_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)
con_test_loader = DataLoader(test_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)

model = MyModel(base_model, config, element_size, X_2_length, batch_size=1)
model.load_state_dict(torch.load(config_2['save_path']))
model.to(device)

# Evaluation mode
model.eval()

accuracy_list = []
macro_f1_list = []
weighted_f1_list = []

# 1. Train part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for input_ids, attention_mask, X_2, y in con_train_loader:
        input_ids, attention_mask, X_2, y = input_ids.to(device), attention_mask.to(device), X_2.to(device), y.to(device)
        outputs = model(input_ids, attention_mask, X_2)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)

print('=====================================================================================================================')
print('Training Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list), '\n')
print('macro_f1: ', f1_score(targets_list, predicted_labels_list, average='macro'))
print('weighted_f1: ', f1_score(targets_list, predicted_labels_list, average='weighted'))
macro_f1_list.append(f1_score(targets_list, predicted_labels_list, average='macro'))
weighted_f1_list.append(f1_score(targets_list, predicted_labels_list, average='weighted'))

# 2. Val part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for input_ids, attention_mask, X_2, y in con_val_loader:
        input_ids, attention_mask, X_2, y = input_ids.to(device), attention_mask.to(device), X_2.to(device), y.to(device)
        outputs = model(input_ids, attention_mask, X_2)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)

print('=====================================================')
print('Val Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list))
print('macro_f1: ', f1_score(targets_list, predicted_labels_list, average='macro'))
print('weighted_f1: ', f1_score(targets_list, predicted_labels_list, average='weighted'))
macro_f1_list.append(f1_score(targets_list, predicted_labels_list, average='macro'))
weighted_f1_list.append(f1_score(targets_list, predicted_labels_list, average='weighted'))

# 3. Test part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for input_ids, attention_mask, X_2, y in con_test_loader:
        input_ids, attention_mask, X_2, y = input_ids.to(device), attention_mask.to(device), X_2.to(device), y.to(device)
        outputs = model(input_ids, attention_mask, X_2)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)

print('=====================================================')
print('Testing Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list))
print('macro_f1: ', f1_score(targets_list, predicted_labels_list, average='macro'))
print('weighted_f1: ', f1_score(targets_list, predicted_labels_list, average='weighted'))
macro_f1_list.append(f1_score(targets_list, predicted_labels_list, average='macro'))
weighted_f1_list.append(f1_score(targets_list, predicted_labels_list, average='weighted'))

print('=====================================================', '\n')
print("Accuracy [Train, Val, Test] ")
print("Macro F1 [Train, Val, Test] ")
print("Weighted F1 [Train, Val, Test]: ")
print(accuracy_list)
print(macro_f1_list)
print(weighted_f1_list, '\n')

accuracy_list_2 = [round(num, 2) for num in accuracy_list]
macro_f1_list_2 = [round(num, 2) for num in macro_f1_list]
weighted_f1_list_2 = [round(num, 2) for num in weighted_f1_list]
print(accuracy_list_2)
print(macro_f1_list_2)
print(weighted_f1_list_2, '\n')
# print('Config: ', config, '\n')
# print('Feature: ', feature)
# print('time_start: ', time_start, 'time_end: ', time_end)